From youtube video: https://www.youtube.com/watch?v=-H-JCgvV0z8

In [2]:
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd

In [3]:
title = "Data Scientist"
location = "San Francisco Bay Area"

In [4]:
list_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%2BScientist&location=San%2BFrancisco%2BBay%2BArea&geoId=90000084&trk=public_jobs_jobs-search-bar_search-submit&start=25"
    
response = requests.get(list_url)    
       
    

In [5]:
response

<Response [200]>

In [6]:
(response.text)

'<!DOCTYPE html>\n\n      <li>\n        \n    \n\n    \n    \n    \n      <div class="base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card" data-entity-urn="urn:li:jobPosting:3935388297" data-impression-id="jobs-search-result-0" data-reference-id="rldal7zsaPDQ7st6ULHhPA==" data-tracking-id="KhRy2KQt5ic5P1HjQ1QxoA==" data-column="1" data-row="26">\n        \n\n        <a class="base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]" href="https://www.linkedin.com/jobs/view/machine-learning-engineer-ads-at-nextdoor-3935388297?position=1&amp;pageNum=2&amp;refId=rldal7zsaPDQ7st6ULHhPA%3D%3D&amp;trackingId=KhRy2KQt5ic5P1HjQ1QxoA%3D%3D" data-tracking-control-name="public_jobs_jserp-result_search-card" data-tracking-client-ingraph data-tracking-will-navigate>\n          \n          <span class="sr-only">\n              \n        \n        Machine Learning Engineer - Ads\n      \n      \n         

In [7]:
job_url ="https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/4043796448?refId=Non6XXO40j%2BdT5s0LxVyhw%3D%3D&trackingId=%2FIOSmTOOfE9LuCI%2B0NQx%2Bg%3D%3D"

response = requests.get(job_url) 

In [8]:
response

<Response [200]>

In [11]:
job_id= "4043796448"

job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"

job_response = requests.get(job_url)

In [13]:
job_response

<Response [200]>

In [ ]:
list_url = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%2BScientist&location=San%2BFrancisco%2BBay%2BArea&geoId=90000084&trk=public_jobs_jobs-search-bar_search-submit&start={page_num}" 




In [3]:
## Creating loop to go through all pages
job_list = []
for page_num in range(1,2):
    list_url = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%2BScientist&location=San%2BFrancisco%2BBay%2BArea&geoId=90000084&trk=public_jobs_jobs-search-bar_search-submit&start={page_num}" 
    # Getting response request from list 
    response = requests.get(list_url)

    list_data = response.text
    list_soup = BeautifulSoup(list_data, 'html.parser')
    page_jobs = list_soup.find_all("li")
    #10 jobs per page
    #print(len(page_jobs))
    
    ## Get job ID's from each page
    id_list = []

    for job in page_jobs:
        base_card_div = job.find("div", {"class": "base-card"})
        job_id = base_card_div.get("data-entity-urn").split(":")[3]
        id_list.append(job_id)
        
    # For every job with ID, get the information
    for job_id in id_list:
        job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
        job_response = requests.get(job_url)
        if(job_response.status_code == 200):
            #print(job_response.status_code)
            job_soup = BeautifulSoup(job_response.text, "html.parser")
            job_post = {}
            try:
                job_post["company_name"] = job_soup.find( "a", {"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
            except: 
                job_post["company_name"] = None
            try:
                job_post["job_title"] = job_soup.find( "h2", {"class": "top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"}).text.strip()
            except: 
                job_post["job_title"] = None
            try:
                job_post["job_description"] = job_soup.find( "div", {"class":"show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden" }).text.strip()
            except: 
                job_post["job_description"] = None
            try:
                job_post["days_ago_posted"] = job_soup.find( "span", {"class": "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
            except:
                job_post["days_ago_posted"] = None
            try:
                job_post["num_applicants"] = job_soup.find( "span", {"class": "num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"}).text.strip()
            except:
                job_post["num_applicants"] = None
            job_list.append(job_post)

    
    

In [4]:
jobs_df = pd.DataFrame(job_list)
jobs_df

,company_name,job_title,job_description,days_ago_posted,num_applicants
0,Notion,"Data Scientist, Product",About UsWe're on a mission to make it possible...,1 week ago,None
1,Netflix,"Machine Learning Engineer Intern, Summer 2025",Netflix is one of the world's leading entertai...,1 month ago,None
2,Google,"Data Scientist, Search Discovery, Research, Se...",Minimum qualifications:Master's degree in Stat...,3 weeks ago,None
3,Fractal,Data Scientist,Data ScientistFractal Analytics is a strategic...,4 days ago,None
4,Fractal,Data Scientist,Data ScientistFractal Analytics is a strategic...,1 week ago,None
5,TikTok,Data Science Graduate (TikTok-Product-Data Sci...,ResponsibilitiesTikTok is the leading destinat...,1 week ago,None
6,Twitch,Data Scientist - Analytics,About UsTwitch is the world’s biggest live str...,3 weeks ago,None
7,LinkedIn,AI Engineer,LinkedIn is the world’s largest professional n...,3 weeks ago,None
8,LinkedIn,Senior Data Scientist - Strategy & Insights,LinkedIn is the world’s largest professional n...,6 days ago,None
9,Databricks,Data Scientist - New Grad (2025 Start),"(P-978)At Databricks, we are passionate about ...",1 month ago,None


In [5]:
jobs_df['job_description'][1]

"Netflix is one of the world's leading entertainment services, with 283 million paid memberships in over 190 countries enjoying TV series, films and games across a wide variety of genres and languages. Members can play, pause and resume watching as much as they want, anytime, anywhere, and can change their plans at any time.Applied Machine Learning Research at Netflix improves various aspects of our business, including personalization algorithms, content generation, system optimization, and innovative tooling for artists. Our research spans many areas of machine learning, including recommender systems, reinforcement learning, computer vision, natural language processing, optimization, and operations research. Great applied research also requires robust machine learning infrastructure, another large area of emphasis at Netflix.Applicants are encouraged to express their interest in one or multiple types of internships listed if your skills and qualifications are aligned.We Are Looking Fo

# Trying out NLP Libraries

In [17]:
from nltk import ngrams
from collections import Counter

result = []
sentence =  'fri evening commute can be long. some people avoid fri evening commute by choosing off-peak hours. there are much less traffic during off-peak. fri evening commute sucks'
#jobs_df['job_description'][1]
# Since you are not considering periods and treats words with - as phrases
sentence = sentence.replace('.', '').replace('-', ' ')

for n in range(len(sentence.split(' ')), 1, -1):
    phrases = []

    for token in ngrams(sentence.split(), n):
        phrases.append(' '.join(token))
    #print(Counter(phrases).most_common(1))

    phrase, freq = Counter(phrases).most_common(1)[0]
    if freq > 1:
        result.append((phrase, n))
        sentence = sentence.replace(phrase, '')

for phrase, freq in result:
    print('%s: %d' % (phrase, freq))

fri evening commute: 3
off peak: 2


In [41]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import ngrams
import nltk.collocations
from nltk import BigramCollocationFinder
from nltk.probability import FreqDist

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [117]:
stop_words = set(stopwords.words("english"))
stop_words.add(':')
stop_words.add(',')
stop_words.add('.')
stop_words.add(', ')
stop_words.add('. ')
stop_words.add('*')
stop_words.add("'")
stop_words.add("'s")
stop_words.add('e.g')
stop_words.add('employees')
stop_words.add('applicants')
stop_words.add(')')
stop_words.add('(')
stop_words.add('eligible')
stop_words.add('participate')



In [118]:
## Tokenizing words

## In this cell, i'm extracting bigrams from one text example (first row of column job_description) and finding the most common 2 word phrases
#nltk.download('punkt')
text = jobs_df['job_description'][6]
tokens = word_tokenize(text)
filtered_list = []
for word in tokens:
    if word.casefold() not in stop_words:
        filtered_list.append(word)

n = 2  # For bigrams

bigrams = list(ngrams(filtered_list, n))
freq_dist = FreqDist(bigrams)
print(freq_dist.most_common(8))
print(type(freq_dist.most_common(8)[1]))

[(('Data', 'Science'), 3), (('product', 'strategy'), 3), (('data', 'scientist'), 2), (('high', 'velocity'), 2), (('Community', 'Health'), 2), (('product', 'improvements'), 2), (('ad', 'hoc'), 2), (('safer', 'rewarding'), 2)]
<class 'tuple'>


In [105]:
n = 3  # For trigrams

trigrams = list(ngrams(filtered_list, n))
freq_dist = FreqDist(trigrams)
print(freq_dist.most_common(5))

[(('product', 'strategy', 'questions'), 2), (('Analytics', 'Data', 'Science'), 2), (('Data', 'Science', 'Computer'), 2), (('Science', 'Computer', 'Science'), 2), (('Computer', 'Science', 'Mathematics'), 2)]


In [119]:

def get_n_grams(word_list, n, top_n):
    '''
        Function to return ngrams
        word_list: pass lowercase word list (filtered for stop words)
        n: the number of words in each phrase (gram)
        top_n: top n number of matches 
    '''
    grams = list(ngrams(word_list, n))
    freq_dist = FreqDist(grams)
    
    topn = freq_dist.most_common(top_n)
    return(topn)


In [125]:
## In this cell I'm going to add the list of common tokens to the dataframe

bigram_list = []
trigram_list = []

#Create a dictionary of the most common bigrams
bigram_dict = {}
trigram_dict = {}

#for every job in job_list dataframe
for job in job_list:
    #Grab the job description
    text = job['job_description']

    #Tokenize and filter unnecessary words, and put relevant words i
    tokens = word_tokenize(text)
    filtered_list = []
    for word in tokens:
        if word.casefold() not in stop_words:
            filtered_list.append(word.lower())  
    
    # For bigrams
    # add bigrams
    top8 = get_n_grams(filtered_list, 2, 8)
    bigram_list.append(top8)
    for tup in top8:
        if tup[0] not in bigram_dict:
            bigram_dict[tup[0]] = tup[1]
        else:
            bigram_dict[tup[0]] += tup[1]

    # add trigrams
    top5 = get_n_grams(filtered_list, 3, 5)
    bigram_list.append(top5)
    for tup in top5:
        if tup[0] not in bigram_dict:
            trigram_dict[tup[0]] = tup[1]
        else:
            trigram_dict[tup[0]] += tup[1]


sorted_dict_bigram = dict(sorted(bigram_dict.items(), key=lambda item: item[1]))
sorted_dict_trigram = dict(sorted(trigram_dict.items(), key=lambda item: item[1]))

print('\n')
print(sorted_dict_bigram)
print('\n')
print(sorted_dict_trigram)



{('minimum', 'qualifications'): 1, ('p-978', 'databricks'): 1, ('databricks', 'passionate'): 1, ('passionate', 'helping'): 1, ('mission', 'make'): 2, ('toolmaking', 'ubiquitous'): 2, ('notion', '’'): 2, ('product', 'data'): 2, ('science', 'team'): 2, ('team', 'work'): 2, ('software', 'engineering'): 2, ('engineering', 'best'): 2, ('best', 'practices'): 2, ('practices', 'version'): 2, ('version', 'control'): 2, ('control', 'testing'): 2, ('testing', 'code'): 2, ('new', 'product'): 2, ('product', 'features'): 2, ('observational', 'experimental'): 2, ('experimental', 'data'): 2, ('base', 'salary'): 2, ('salary', 'range'): 2, ('data', 'scientistfractal'): 2, ('scientistfractal', 'analytics'): 2, ('tiktok', 'mission'): 2, ('mission', 'inspire'): 2, ('analysis', 'experimentation'): 2, ('reviewed', 'rolling'): 2, ('data', 'scientist'): 2, ('high', 'velocity'): 2, ('community', 'health'): 2, ('product', 'improvements'): 2, ('ad', 'hoc'): 2, ('safer', 'rewarding'): 2, ('role', 'based'): 2, ('

import gensim
from gensim import corpora

documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time"]

texts = [doc.split() for doc in documents]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word=dictionary, passes=15)
print(lda.print_topics(num_topics=3, num_words=3))

In [30]:
# Installation
#!pip3 install pytextrank

import spacy
import pytextrank


In [39]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

# change this to read in your data
finder = BigramCollocationFinder.from_words(jobs_df['job_description'][4])
# only bigrams that appear 3+ times
finder.apply_freq_filter(3)

# return the 10 n-grams with the highest PMI
finder.nbest(bigram_measures.pmi, 10)

[('0', '0'),
 ('A', 'I'),
 ('.', 'E'),
 ('q', 'u'),
 ('.', 'P'),
 ('x', 'p'),
 ('T', 'h'),
 ('Y', 'o'),
 ('F', 'r'),
 ('C', 'o')]